In [1]:
!nvidia-smi

Thu Apr 25 04:15:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:01:00.0 Off |                  N/A |
|  0%   38C    P2             118W / 420W |   1915MiB / 24576MiB |     19%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
import os

# # Check if CUDA is available
# cuda_available = torch.cuda.is_available()
# print(f"CUDA available: {cuda_available}")

# # Set the device to GPU if CUDA is available
# device = torch.device("cuda" if cuda_available else "cpu")
# print(f"Using device: {device}")

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="1" #select 3090 Super

if torch.cuda.device_count() >= 2:
    print(f"{torch.cuda.device_count()} GPUs are available")

num_gpus = torch.cuda.device_count()
for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

device = torch.device("cuda:0")  # Default GPU
    
# # Example of moving a tensor to GPU
# if cuda_available:
#     #Get ID
#     gpu_id = torch.cuda.current_device()
#     print(f"GPU ID being used: {gpu_id}")

#     # Get the name of the current GPU
#     gpu_name = torch.cuda.get_device_name(gpu_id)
#     print(f"GPU Name: {gpu_name}")

# else:
#     print("CUDA is not available. No GPU is being used.")

GPU 0: NVIDIA GeForce RTX 2060 SUPER


In [3]:
import gc
torch.cuda.empty_cache()
gc.collect()

12

In [4]:
def print_gpu_memory():
    allocated = torch.cuda.memory_allocated()
    max_allocated = torch.cuda.max_memory_allocated()
    print(f"Current memory allocated: {allocated / 1024**3:.2f} GB")
    print(f"Max memory allocated during this session: {max_allocated / 1024**3:.2f} GB")

In [5]:


#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching_allocator"

# Load Libraries

In [6]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

# Load Data

In [7]:
#Read smote data back to nb ad cuda df
X_train = pd.read_parquet("../capstone_data/archive/SMOTE_train_test_split/X_train_smote.parquet")
y_train = pd.read_parquet("../capstone_data/archive/SMOTE_train_test_split/y_train_smote.parquet")['label']
X_test = pd.read_parquet("../capstone_data/archive/SMOTE_train_test_split/X_test_smote.parquet")
y_test = pd.read_parquet("../capstone_data/archive/SMOTE_train_test_split/y_test_smote.parquet")['label']

print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


# Standardize numerical Cols

In [8]:
# Scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


numeric_columns = ['duration', #extremely right skewed
                   'orig_bytes', #extremely right skewed
                   'resp_bytes', #extremely right skewed
                   'orig_pkts', #extremely right skewed
                   'resp_pkts', #extremely right skewed
                   'orig_ip_bytes', ##extremely right skewed
                   'resp_ip_bytes'] #extremely right skewed

#Log transformation for extremely right skewed distributions
constant = 1e-6
X_train[numeric_columns] = X_train[numeric_columns].apply(lambda x: np.log(x + constant))
X_test[numeric_columns] = X_test[numeric_columns].apply(lambda x: np.log(x + constant))

# Convert to NumPy Array then create tensors

In [9]:
# #convert to numpy array
# X_train_np = X_train.to_numpy()
# X_train_np = X_train_np.reshape(X_train_np.shape[0], 1, X_train_np.shape[1])
# y_train_np = y_train.to_numpy()
# X_test_np = X_test.to_numpy()
# y_test_np = y_test.to_numpy()

# # Convert to tensors suitable for PyTorch
# X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # Reshape for BCEWithLogitsLoss

# X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)  # Reshape for BCEWithLogitsLoss

In [10]:
# #Create TensorDatasets
# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# # Create DataLoaders
# batch_size = 32
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Set RNN:LSTM Model

In [11]:
# Max memory allocated during this session: 9.69 GB
# 106.60491490364075
# Epoch [10/10], Loss: 0.6622
# Accuracy of the model on the test data: 67.19%


# # Convert to tensors suitable for PyTorch
# X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
# y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)  # Reshape for BCEWithLogitsLoss
# X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
# y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)  # Reshape for BCEWithLogitsLoss

# # Create Tensor datasets
# train_data = TensorDataset(X_train_tensor, y_train_tensor)
# test_data = TensorDataset(X_test_tensor, y_test_tensor)

# # Create DataLoaders
# batch_size = 1228800  # You can adjust this based on your GPU memory
# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


# class LSTMClassifier(nn.Module):
#     def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
#         super(LSTMClassifier, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.layer_dim = layer_dim
#         self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):
#         # Check if the input tensor is 2D and if so, unsqueeze it to 3D
#         if x.dim() == 2:
#             x = x.unsqueeze(1)  # Add a sequence dimension

#         h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
#         c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
#         out, _ = self.lstm(x, (h0, c0))
#         out = self.fc(out[:, -1, :])  # Assuming that you want the last time step
#         return out

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = LSTMClassifier(input_dim=X_train.shape[1], hidden_dim=50, layer_dim=1, output_dim=1)
# model = model.to(device)

# criterion = nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# num_epochs = 10
# for epoch in range(num_epochs):
#     start_time = time.time()
#     model.train()
#     total_loss = 0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()

#     print_gpu_memory()
#     print(time.time() - start_time)
    
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}')

# # Evaluate the model
# model.eval()
# correct = 0
# total = 0
# with torch.no_grad():
#     for inputs, labels in test_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         outputs = model(inputs)
#         predicted = torch.sigmoid(outputs) > 0.5
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
# accuracy = 100 * correct / total
# print(f'Accuracy of the model on the test data: {accuracy:.2f}%')

In [12]:
%%time

import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler

def df_to_tensor(df):
    return torch.tensor(df.values, dtype=torch.float32).to(device)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).unsqueeze(1).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).unsqueeze(1).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Create Tensor datasets
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

# Hyperparameters
batch_size = 64 #524288 for 3090rtx
train_loader = DataLoader(train_data, 
                          batch_size=batch_size, 
                          shuffle=True
                          )
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)



# Define the LSTM Model
class BinaryLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BinaryLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.5 if num_layers > 1 else 0)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        output, (hn, _) = self.lstm(x)
        # Handle cases where the LSTM output directly corresponds to hidden states without additional sequence dimension
        last_time_step = output[:, -1, :] if output.dim() == 3 else hn.squeeze(0)
        out = self.linear(last_time_step)
        return torch.sigmoid(out)

# Train and evaluate function
def train_and_evaluate(model, train_loader, test_loader, optimizer, criterion, num_epochs):
    best_accuracy = 0
    best_model = None
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        total_loss = 0
        total_batches = 0
        for i, (inputs, labels) in enumerate(train_loader):
            model.zero_grad(set_to_none=True)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_batches += 1
            loss.backward()
            optimizer.step()
            #print(f'Epoch {epoch+1}, Iteration {i+1}, Loss: {loss.item()}')

            # Print loss for each epoch iteration
        print(f'Epoch {epoch+1}, Batch {total_batches}: Last Loss {loss.item():.4f}')

        average_loss = total_loss / total_batches
        print(f'Epoch {epoch+1}: Average Training Loss {average_loss:.4f}')

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                loss = criterion(outputs, labels)  # Calculate loss
                total_loss += loss.item()
            val_loss = total_loss / len(test_loader)
            scheduler.step(val_loss)

        accuracy = correct / total
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            print(f'New best accuracy {best_accuracy:.4f}, Epoch {epoch+1}: ')

        

        print(f'Epoch {epoch+1} Time: {time.time() - start_time:.2f} seconds')

    return best_model, best_accuracy

# Hyperparameters
input_size = X_train.shape[1] #number of features
hidden_sizes = [64,128,256]
num_layers_list = [2,3]
learning_rates = [0.000001]
num_epochs = 50

# Grid search for best hyperparameters
for hidden_size in hidden_sizes:
    for num_layers in num_layers_list:
        for learning_rate in learning_rates:            
            model = BinaryLSTM(input_size, hidden_size, num_layers)
            model.cuda()


            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            criterion = nn.BCELoss()
            #LR Scheduleer
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
            total_loss = []
            best_model, best_accuracy = train_and_evaluate(model, train_loader, test_loader, optimizer, criterion, num_epochs)            
            print(f'Best Model with hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, accuracy={best_accuracy:.4f}')

Epoch 1, Batch 205791: Last Loss 0.6673
Epoch 1: Average Training Loss 0.6888
New best accuracy 0.5872, Epoch 1: 
Epoch 1 Time: 385.14 seconds
Epoch 2, Batch 205791: Last Loss 0.6742
Epoch 2: Average Training Loss 0.6662
New best accuracy 0.6584, Epoch 2: 
Epoch 2 Time: 391.49 seconds
Epoch 3, Batch 205791: Last Loss 0.6015
Epoch 3: Average Training Loss 0.6490
New best accuracy 0.6728, Epoch 3: 
Epoch 3 Time: 387.28 seconds
Epoch 4, Batch 205791: Last Loss 0.4763
Epoch 4: Average Training Loss 0.6392
Epoch 4 Time: 389.54 seconds
Epoch 5, Batch 205791: Last Loss 0.5946
Epoch 5: Average Training Loss 0.6320
Epoch 5 Time: 388.45 seconds
Epoch 6, Batch 205791: Last Loss 0.6211
Epoch 6: Average Training Loss 0.6295
Epoch 6 Time: 389.40 seconds
Epoch 7, Batch 205791: Last Loss 0.5654
Epoch 7: Average Training Loss 0.6273
New best accuracy 0.6790, Epoch 7: 
Epoch 7 Time: 389.09 seconds
Epoch 8, Batch 205791: Last Loss 0.7956
Epoch 8: Average Training Loss 0.6266
Epoch 8 Time: 388.06 seconds



KeyboardInterrupt



In [13]:
torch.save(best_model.state_dict(), 'best_lstm_model.pth')